In [2]:
import pandas as pd
import glob  
import sys
import os
import time
import re
import numpy as np
from tqdm import tqdm

# Get Data with filtering

Get keyword list

In [3]:
keyword_list = pd.read_excel("../data/Text_keyword_0620.xlsx")

In [4]:
print(len(keyword_list))
keyword_list.head()

65


,Unnamed: 0,Target 키워드
0,1,인구
1,2,도시
2,3,도심
3,4,지역
4,5,지자체


In [5]:
DATA_PATH = '../data/pkl/'


df_filter = pd.DataFrame()

df_len = []
city_list = []
total_before_len = 0

for f in tqdm(glob.glob(f'{DATA_PATH}*.pkl')):
    if '시' in f:
        city = re.search(f'{DATA_PATH}(.+?)_merged.pkl', f).group(1)
        df = pd.read_pickle(f)
        df['city'] = city
        city_list.append(city)
        city_before_len = len(df)
        total_before_len += city_before_len
        filter_keyword = []
        for idx, keyword in enumerate(df['키워드']):
            keyword =  str(keyword).split(',')
            filter_keyword.append(set(keyword).intersection(set(keyword_list['Target 키워드'])))
        df['filter_keyword'] = filter_keyword
        filtered_df = df.loc[df['filter_keyword'].map(len) != 0]
        #print(f"City = {city}, before len = {city_before_len}, after len = {len(filtered_df)}")
        df_filter = pd.concat([df_filter, filtered_df])        
        temp = [city,  city_before_len]
        df_len.append(temp)
pd.DataFrame(df_len, columns=['city', 'count']).to_csv("../result/before_filter_city_count.csv")

#print(f"before : {total_before_len}, after : {len(df_filter)}")

100%|██████████| 81/81 [06:08<00:00,  4.55s/it]


In [6]:
df_filter.columns

Index(['뉴스 식별자', '일자', '언론사', '기고자', '제목', '통합 분류1', '통합 분류2', '통합 분류3',
       '사건/사고 분류1', '사건/사고 분류2', '사건/사고 분류3', '인물', '위치', '기관', '키워드',
       '특성추출(가중치순 상위 50개)', '본문', 'URL', '분석제외 여부', 'city', 'filter_keyword'],
      dtype='object')

### 분석제외 여부에 따른 filtering

In [7]:
df_filter['분석제외 여부'].value_counts()

중복        178497
예외        134544
중복, 예외      4648
Name: 분석제외 여부, dtype: int64

In [8]:
df_filter.shape

(6305834, 21)

In [9]:
df_filter2 = df_filter[df_filter['분석제외 여부'].isnull()]

In [10]:
print(df_filter2['분석제외 여부'].value_counts())
df_filter2.shape

Series([], Name: 분석제외 여부, dtype: int64)


(5988145, 21)

### 위치 정보에 너무 많은 도시가 등장하면 제거하기

In [11]:
df_filter2.reset_index(inplace=True, drop = True)

In [12]:
def find_general_news(article):
    related_city = []
    article = str(article).split(',')
    city = [x for i in article for x in city_list if i in x]
    related_city.append(city)
    return related_city


In [13]:
related_city = df_filter2['위치'].apply(lambda x : find_general_news(x))

In [14]:
df_filter2['related_city'] = related_city

/tmp/ipykernel_17364/3785826648.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filter2['related_city'] = related_city


In [15]:
idx_list = []
for idx, city in enumerate(related_city):
    if len(city[0]) <5:
        idx_list.append(idx)

In [16]:
df_filter3 = df_filter2.iloc[idx_list]

In [17]:
len(df_filter3)

5246886

### 통합 분류1을 기준으로 필터링

In [18]:
df_filter3.columns

Index(['뉴스 식별자', '일자', '언론사', '기고자', '제목', '통합 분류1', '통합 분류2', '통합 분류3',
       '사건/사고 분류1', '사건/사고 분류2', '사건/사고 분류3', '인물', '위치', '기관', '키워드',
       '특성추출(가중치순 상위 50개)', '본문', 'URL', '분석제외 여부', 'city', 'filter_keyword',
       'related_city'],
      dtype='object')

In [19]:
df_filter3['통합 분류1'].value_counts()

지역>강원            419483
지역>충남            418902
지역>경기            410381
지역>경남            405968
지역>충북            282363
                  ...  
스포츠>축구>국가대표팀        385
경제>외환               210
스포츠>야구>메이저리그        106
스포츠>야구>일본프로야구        20
006000000             1
Name: 통합 분류1, Length: 87, dtype: int64

In [20]:
sum(df_filter3['통합 분류1'] == '정치>북한')
sum(df_filter3['통합 분류1'] == '정치>국회_정당')
sum(df_filter3['통합 분류1'] == '정치>선거')
sum(df_filter3['통합 분류1'] == '정치>외교')

7383

In [21]:
'국제>예시' == re.compile("^국제")

False

In [22]:
import re

idx_lst = []
for idx, cat in enumerate(df_filter3['통합 분류1']):
    if re.compile('^국제').findall(cat):
        continue
    elif re.compile('^스포츠').findall(cat):
        continue
    elif re.compile('^정치>북한').findall(cat):
        continue
    elif re.compile('^정치>국회_정당').findall(cat):
        continue
    elif re.compile('^정치>선거').findall(cat):
        continue
    elif re.compile('^정치>외교').findall(cat):
        continue
    else:
        idx_lst.append(idx)

In [37]:
df_filter4 = df_filter3.iloc[idx_lst]
df_filter4.reset_index(inplace = True, drop = True)

In [40]:
df_filter4

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,...,위치,기관,키워드,특성추출(가중치순 상위 50개),본문,URL,분석제외 여부,city,filter_keyword,related_city
0,1500701.201604,19911212,부산일보,부산일보,[사람],지역>대전,지역>부산,지역>충남,NaN,NaN,...,"전남,최고경영자,凡一6동","金泉시의회,알래스카간,KBS홀,여성단체협의회장,동력자원부,모금추진위원회,학교,농림수...","사람,국제통상,주제,강연,전무,金殷湘,한국무역협회,11일,6시,강의실,부산대,경영,...","격려금,손목시계,국제통상,동구,원씩,慶南,金泉,머코스키,회의실,釜山日報社,근로자들,...",‘국제통상’ 주제 강연\n◇金殷湘 한국무역협회 전무=11일 오후 6시40분 부산대 ...,NaN,NaN,김해시,"{지역, 사업, 개최}",[[]]
1,1100611.201604,19911208,서울신문,NaN,농수산물 음성정보/74개 시 군으로 확대,IT_과학>인터넷_SNS,IT_과학>과학,IT_과학>모바일,NaN,NaN,...,"김해시,홍성군,서울,부산","농수산물유통공사,농수산","농수산물,음성,정보,74개,확대,농수산물유통공사,7일,농수산물,정보,음성,서비스,제...","농수산물,부산,김해시,전자식,부산등,전자식전화기,홍성군등,전화기,이용자,서울,농수산...",농수산물유통공사는 7일부터 농수산물정보음성서비스 제공지역을 서울 부산등 15개 시에...,http://www.seoul.co.kr/news/newsView.php?id=19...,NaN,김해시,{지역},[[김해시]]
2,1100611.201604,19911206,서울신문,NaN,뇌물 받고 서류 위조/피해보상금 받게해/공무원등 둘 구속,지역>부산,사회>사건_사고,지역>강원,범죄>정치>뇌물수수,범죄>범죄일반>사기,...,"김해시,외동,구포1동,북구,경남,부산","부산지검,김해군청,특수부","뇌물,서류,위조,피해보상금,공무원등,구속,부산,부산지검,특수,임태성,검사,뇌물,허위...","축정계,공무원,보상금,부산,부산지검,임태성,김해군청,경남,어업피해보상금,김해시,외동...",【부산】 부산지검 특수부 임태성 검사는 5일 뇌물을 받고 허위어업허가자 확인신청서를...,http://www.seoul.co.kr/news/newsView.php?id=19...,NaN,김해시,{산업},[[김해시]]
3,1100701.199112,19911206,세계일보,김기현,허위서류 꾸며 보상금/뇌물 받은 공무원 구속,지역>부산,사회>사건_사고,지역>전북,범죄>정치>뇌물수수,범죄>범죄일반>사기,...,"김해시,부산시,낙동강,외동,구포1동,북구,경남,남원","부산지검,검찰,김해군청","허위서류,보상금,뇌물,공무원,구속,임태성검사,부산지검,특수,임태성,검사,뇌물,허위,...","보상금,공무원,김해군청,손씨,부산지검,낙동강,김상수,축정계,신청서,김씨,어업피해보상...",【부산=김기현기자】 부산지검 특수부 임태성검사는 5일 뇌물을 받고 허위어업허가자 확...,NaN,NaN,김해시,"{지역, 주민, 산업}","[[김해시, 남원시]]"
4,1100901.199111,19911128,중앙일보,NaN,청와대 비서 사칭 백90억 사기 미수,사회>사건_사고,사회>노동_복지,정치>정치일반,범죄>범죄일반>사기,NaN,...,"김해시,대성동,10만3천평,서울,부산,쌍림동,(주)대도건설","정부,경찰청,청와대,특수대","청와대,비서,사칭,사기,미수,영장,3명,수배,경찰청,특수대,비서관,청와대,행세,건설...","청와대,3명,부동산,김해시,비서관,부산,이씨,대도건설,40억,계약금,서울,이일용,김...",경찰청 특수대는 28일 청와대 비서관 등의 행세를 하면서 건설업체를 상대로 구획정리...,https://www.joongang.co.kr/article/2663652,NaN,김해시,{사업},[[김해시]]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4964635,1200101.201604,20140103,경기일보,박상돈 기자,현안 전과정 공개 ‘소통시정’,미분류,NaN,NaN,NaN,NaN,...,"연풍,운정,문산","파주시,파주시청","현안,현안,공개,소통,시정,파주시,현안,시정,투자,사업,사업,입안,시민,공개,사업내...","홈페이지,정책실명제,10억,추진과정,43건,이력서,파주시청,장기간,책임소재,도로확,...",파주시는 2일 시정 현안 및 대규모 투자사업 등 주요 사업에 대해 입안부터 완료까지...,http://www.kyeonggi.com/news/articleView.html?...,NaN,파주시,"{주민생활, 사업, 도시, 건설}",[[]]
4964636,1400501.201401,20140102,중부일보,NaN,대규모 산업단지 조성사업 엇갈린 운명 '시장 뜻따라',지역>경기,지역>전북,경제>부동산,NaN,NaN,...,"피해,경기,파주,도일동,파주시,운정3지구,평택시,평택,법원읍,평택브레인시티,경기도","news,평택시장,파주시장,평택브레인시티통합지주협의회,평택브레인시티개발주식회사,법원...","조성사업,산업,단지,조성,사업,운명,시장,평택브레인시티,조감도,수년,존폐,수년,기로...","경기도,파주,브레인시티,평택시,파주시,평택,사업기간,만큼,3천,관계자,산업단지,시행...",평택브레인시티 조감도 \n\n수년 간 존폐의 기로에 서서 위태롭게 추진돼온 대규모 ...,www.joongboo.com/?mod=news&act=articleView&idx...,NaN,파주시,"{피해, 사업, 조성, 유치, 주민, 주민들, 산업}","[[파주시, 파주시, 평택시, 평택시]]"
4964637,1200101.201604,20140102,경기일보,이호준 기자,"[지방자치, 해법을 묻다] 임창열 민선 2기 경기지사",정치>정치일반,NaN,NaN,NaN,NaN,...,"세종시,서울,대중국,경기,한류월드,평택항,경기도,수도권,경기북,시흥,경기지사,경기지...","지역신용보증재단법,서울대,명지대학교대학원,평택항,미래산업,LG필립스 산업단지,경기도...","임창열,민선,경기,지사,국무,회의,참석,차관,도지사,도민,자존심,상처,세종시장,인구...","경기도,경기지사,지자체,고양시,수도권,임창열,대통령령,판교테크노밸리,imf,박근혜,...","국무회의 참석 못하는 차관급 도지사, 도민 자존심에 상처\n\n인구 20만 세종시장...",http://www.kyeonggi.com/news/articleView.html?...,NaN,파주시,"{사업, 지자체, 조성, 인프라, 유치, 개최, 지역, 주민, 인구, 관광, 산업,...","[[시흥시, 파주시, 의정부시, 고양시]]"
4964638,1200201.201401,20140101,경인일보,NaN,2013년 마지막까지 줄이은 화마,사회>사건_사고,사회>사회일반,NaN,사고>산업사고>화재,NaN,...,"곤지암읍,경기도내,파주,광주시,파주시,광주","금촌동,신촌리,경찰,국립과학수사연구원","마지막,화마,31일,경기도내,발생,화재,3명,목숨,이날,2시,파주시,금촌동,원룸,발...","이씨,광주시,곤지암읍,이윤희기자,경기도내,이종태,광주,파주,연구원,국립과학수사연구원...",31일 새벽 경기도내 곳곳에서 발생한 화재로 모두 3명이 목숨을 잃었다.\n\n이날...,http://www.kyeongin.com/news/articleView.html?...,NaN,파주시,"{피해, 발생}","[[파주시, 광주시, 파주시, 광주시]]"


In [41]:
len(df_filter4)

4964640

### Count # by city

In [25]:
df_after_filter_len = pd.DataFrame(df_filter4['city'].value_counts())

In [26]:
df_after_filter_len.reset_index(inplace = True)

In [27]:
df_len = pd.DataFrame(df_len)

In [28]:
df_len.columns = ['city', 'before']
df_after_filter_len.columns = ['city', 'after']

In [29]:
city_count = pd.merge(df_len, df_after_filter_len)

In [30]:
city_count['reduced ratio'] = city_count['after'] / city_count['before']

In [31]:
city_count.to_csv("../result/city_count_before_and_after.csv")

save

In [33]:
print(df_filter4.shape)
df_filter4.to_pickle(f"{DATA_PATH}merged_filtered_0630.pkl")
print('done well')

(6305834, 21)
done well


In [5]:
DATA_PATH = '../data/pkl/'
df_filter4 = pd.read_pickle(f"{DATA_PATH}merged_filtered_0630.pkl")

In [6]:
empty_dummy = pd.DataFrame(
    index = range(0, len(df_filter4)),
    columns = keyword_list['Target 키워드']
    )

In [53]:
empty_dummy.loc[0,'지역']

nan

In [7]:
for idx, keywords in tqdm(enumerate(df_filter4.filter_keyword)):
    for token in keywords:
        empty_dummy.loc[idx, token] = 1

4964640it [06:46, 12228.05it/s]


In [8]:
empty_dummy = empty_dummy.fillna(0)
empty_dummy

Target 키워드,인구,도시,도심,지역,지자체,마을,쇠퇴,감소,활성화,도시발전,...,공공시설,생활시설,기반시설,인프라,재개발,재건축,공동화,조성,건설,공원화
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4964635,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4964636,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4964637,1,0,0,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
4964638,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
df_filter4.reset_index(inplace=True, drop = True)

In [16]:
df_keyword_dummy = pd.concat([df_filter4, empty_dummy], axis = 1)

In [19]:
df_keyword_dummy.to_pickle("../data/df_filter_dummy.pkl")

In [22]:
df_keyword_dummy[df_keyword_dummy['city'] == '포항시'].to_excel('../data/df_filter_dummy_pohang.xlsx')